<a href="https://colab.research.google.com/github/anselmo-pitombeira/Notebooks/blob/master/Cartpole_Q_Learning_Linear_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gym
import numpy as np
import numpy.random as rd


def feature_vector(state, a):
    ##build feature vector
    phi = np.zeros(9)
    phi[0] = 1
    if a == 0:
        phi[1:5] = state
    else:
        phi[5:] = state
        
    return phi


def q(phi, w):
    ##x - feature vector
    return w.dot(phi)

def greedy_action(state, w):
    phi1 = feature_vector(state, 0)
    phi2 = feature_vector(state, 1)
    q1 = q(phi1, w)    ##Value of action 0
    q2 = q(phi2, w)     ##Value of action 1


    if q1 > q2:
        action = 0
#        print("Action=", 0)
    else:
        action = 1
#        print("Action=", 1)
    return action

max_ang = 12
alfa = 0.30
gamma = 0.999
epsilon = 1.0
actions = [0,1]

env = gym.make('CartPole-v1')
env.env.theta_threshold_radians=np.radians(max_ang)
env.seed(111)

w = np.zeros(9)    ##initialize state
tt = 0

for i_episode in range(1,1000):

    print('Episódio ', i_episode)
    state = env.reset()
    if rd.uniform() < 1-epsilon: ##Takes greedy action with probability 1-epsilon
        action = greedy_action(state, w)
    else:
        action = env.action_space.sample()
    t = 0
    
    while True:

        if i_episode % 10 ==0:
            env.render()
#        
        ##build feature vector
        phi = feature_vector(state, action)
        ##Pick next state
        next_state, reward, done, info = env.step(action)
        ##take greedy action
        if rd.uniform() < 1-epsilon: ##Takes greedy action with probability 1-epsilon
            next_action = greedy_action(state, w)
        else:
            next_action = env.action_space.sample()

        ##build next feature vector
        phi_next = feature_vector(next_state, next_action)

        ##update weights vector
        delta = reward+gamma*q(phi_next, w)-q(phi, w)
        w = w+alfa*(delta)*phi

        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break
        t+=1
        tt+=1
        
        state = next_state
        action = next_action

        alfa = np.log(t+i_episode)/(t+i_episode)
        epsilon = (99/(100+tt))


    print('e = ', epsilon)
    print('alfa=',alfa)
    
env.close()